Multi AI Agent Architecture

In [28]:
from typing import Annotated, TypedDict, List, Literal, Dict, Any
from langgraph.graph import StateGraph,START,END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.prebuilt import tools_condition
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
import os
from dotenv import load_dotenv



In [4]:
load_dotenv()

True

In [5]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [8]:
class AgentState(MessagesState):
    next_agent:str
    

In [9]:
@tool
def search_web(query:str) -> str:
    """Search the web for information"""
    search = TavilySearchResults(max_results=3)
    results = search.invoke(query)
    return str(results)

@tool
def write_summary(content:str)-> str:
    """Write a summary of the content"""

    summary  = f"Summary of the content is : \n\n{content}"
    return summary

In [18]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("groq:llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001A46C4BE870>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A46BCBE720>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
def researcher_agent(state:AgentState):
    """Researcher agent that searched the content"""
    messages = state["messages"]
    system_msg = SystemMessage(content="You are a researcher agent. Use the search_web tool to find the requested information")

    researcher_llm = llm.bind_tools([search_web])
    response = researcher_llm.invoke([system_msg]+messages)

    return {
        "messages":[response],
        "next_agent":"writer"
    }

In [12]:
def writer_agent(state:AgentState):
    """Writer agent that writes the content"""
    messages = state["messages"]
    system_msg = SystemMessage(content="You are a technical writer agent. Use the information and write content like a blog")
    response = llm.invoke([system_msg]+messages)

    return {
        "messages":[response],
        "next_agent":"end"
    }

In [13]:
def execute_tool(state:AgentState):
    """Execute any pending tool calls"""

    messages = state["messages"]
    last_message = messages[-1]

    if hasattr(last_message,"tool_calls") and last_message.tool_calls:

        tool_node = ToolNode([search_web, write_summary])
        response = tool_node.invoke(state)
        return response
    
    return state

In [15]:
def multi_agent():
    """Multi Agent system creator"""

    workflow = StateGraph(MessagesState)

    workflow.add_node("researcher", researcher_agent)
    workflow.add_node("writer", writer_agent)

    workflow.set_entry_point("researcher")
    workflow.add_edge("researcher", "writer")
    workflow.add_edge("writer", END)
    final_workflow = workflow.compile()

    return final_workflow

In [16]:
final_workflow = multi_agent()

In [22]:
response = final_workflow.invoke({"messages":"Research about the use case of Agentic AI in bussiness"})

In [26]:
response["messages"][-1].content

" \n\n**Unlocking the Power of Agentic AI in Business: Exploring its Use Cases**\n\nArtificial Intelligence (AI) has revolutionized the way businesses operate, and Agentic AI is a subset of AI that has gained significant attention in recent years. Agentic AI, also known as autonomous AI or self-aware AI, is designed to make decisions and take actions on its own, mimicking human-like behavior. In this blog post, we'll explore the use cases of Agentic AI in business and how it can transform your organization.\n\n**What is Agentic AI?**\n\nAgentic AI is a type of AI that possesses the ability to reason, learn, and adapt to its environment, making decisions and taking actions that are aligned with its goals and objectives. This type of AI is based on the concept of agency, which refers to the ability of an entity to act independently and make decisions that affect its environment.\n\n**Business Use Cases of Agentic AI**\n\n1. **Personalized Customer Experience**: Agentic AI can analyze cus

Supervisor Multi Agent

In [ ]:
class SupervisorState(MessagesState):
    """State for the supervisor multi-agent system"""
    current_agent:str = ""
    task_assignments:Dict[str,List[str]] = {}
    agents_outputs : Dict[str,Any] = {}
    workflow_stage:str = "initial"
    iteration_count:int=0
    max_iteration:int=10
    final_output : str = ""